本方案直接对每个回归头输出的proposal裁剪出图像块，输入CLIP

本方案不需要额外的训练

In [1]:
import torch
import torch.nn as nn

import pickle

from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

from transformers import CLIPProcessor, CLIPModel

from PIL import Image

test_mode = True

F:\SomeRandomTask\CLIP-MRNN\.venv\lib\site-packages\detectron2\model_zoo\model_zoo.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


首先读取Mask R-CNN，并将其分类头去掉

In [2]:
MRCNN_PATH = "../model/model_final_f10217.pkl"
CONFIG_FILE = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"

In [3]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(CONFIG_FILE))
cfg.MODEL.WEIGHTS = MRCNN_PATH
cfg.MODEL.DEVICE = "cuda"

In [4]:
mask_mrcnn_model = build_model(cfg)
mask_mrcnn_model.eval()
DetectionCheckpointer(mask_mrcnn_model).load(cfg.MODEL.WEIGHTS)

{'__author__': 'Detectron2 Model Zoo'}

In [5]:
if test_mode:
    for name, param in mask_mrcnn_model.named_parameters():
        print(name, param.shape)

backbone.fpn_lateral2.weight torch.Size([256, 256, 1, 1])
backbone.fpn_lateral2.bias torch.Size([256])
backbone.fpn_output2.weight torch.Size([256, 256, 3, 3])
backbone.fpn_output2.bias torch.Size([256])
backbone.fpn_lateral3.weight torch.Size([256, 512, 1, 1])
backbone.fpn_lateral3.bias torch.Size([256])
backbone.fpn_output3.weight torch.Size([256, 256, 3, 3])
backbone.fpn_output3.bias torch.Size([256])
backbone.fpn_lateral4.weight torch.Size([256, 1024, 1, 1])
backbone.fpn_lateral4.bias torch.Size([256])
backbone.fpn_output4.weight torch.Size([256, 256, 3, 3])
backbone.fpn_output4.bias torch.Size([256])
backbone.fpn_lateral5.weight torch.Size([256, 2048, 1, 1])
backbone.fpn_lateral5.bias torch.Size([256])
backbone.fpn_output5.weight torch.Size([256, 256, 3, 3])
backbone.fpn_output5.bias torch.Size([256])
backbone.bottom_up.stem.conv1.weight torch.Size([64, 3, 7, 7])
backbone.bottom_up.res2.0.shortcut.weight torch.Size([256, 64, 1, 1])
backbone.bottom_up.res2.0.conv1.weight torch.Size

去掉：

roi_heads.box_predictor.cls_score.weight torch.Size([81, 1024])

roi_heads.box_predictor.cls_score.bias torch.Size([81])

构建MyMaskRCNN类来实现

In [6]:
class MyMaskRCNN(nn.Module):
    def __init__(self, cfg=None, original_model=None):
        """
        若提供 original_model：直接复制结构并去掉分类头；
        若提供 cfg：根据配置文件构造同结构模型，再去掉分类头；
        """
        super().__init__()
        if original_model is not None:
        #     print("[MyMaskRCNN] 从原始模型复制结构。")
            self.backbone = original_model.backbone
            self.proposal_generator = original_model.proposal_generator
            self.roi_heads = original_model.roi_heads
        elif cfg is not None:
        #     print("[MyMaskRCNN] 根据 cfg 创建空模型结构。")
            model = build_model(cfg)
            self.backbone = model.backbone
            self.proposal_generator = model.proposal_generator
            self.roi_heads = model.roi_heads
        else:
            raise ValueError("必须提供 original_model 或 cfg 之一。")
        
        # 去掉不需要的分类头
        if hasattr(self.roi_heads, "box_predictor"):
            box_predictor = self.roi_heads.box_predictor
            if hasattr(box_predictor, "cls_score"):
                del box_predictor.cls_score
                box_predictor.cls_score = None

    def save_to_pkl(self, path):
        """
        将模型参数保存为.pkl文件
        """
        state_dict = self.state_dict()
        data = {
            "model": state_dict,
            "__author__": "Task 1",
            "matching_heuristics": True
        }
        with open(path, "wb") as f:
            pickle.dump(data, f)
        # print(f"[MyMaskRCNN] 参数已保存到 {path}")

    def load_from_pkl(self, path):
        """
        从.pkl文件加载模型参数
        """
        with open(path, "rb") as f:
            data = pickle.load(f)
            
        state_dict = data["model"] if "model" in data else data
        missing, unexpected = self.load_state_dict(state_dict, strict=False)
        # print(f"[MyMaskRCNN] 从 {path} 加载完成。")
        # if missing:
        #     print("未加载参数：", missing)
        # if unexpected:
        #     print("未使用参数：", unexpected)

    def forward(self, *args, **kwargs):
        raise NotImplementedError()



In [8]:
MyMRCNN_PATH = "../model/my_mask_rcnn.pkl"

my_mask_rcnn_model = MyMaskRCNN(original_model=mask_mrcnn_model)
my_mask_rcnn_model.eval()
my_mask_rcnn_model.save_to_pkl(MyMRCNN_PATH)

[MyMaskRCNN] 从原始模型复制结构。
[MyMaskRCNN] 参数已保存到 ../model/my_mask_mrcnn.pkl


测试

In [9]:
if test_mode:
    for name, param in my_mask_rcnn_model.named_parameters():
        print(name, param.shape)

backbone.fpn_lateral2.weight torch.Size([256, 256, 1, 1])
backbone.fpn_lateral2.bias torch.Size([256])
backbone.fpn_output2.weight torch.Size([256, 256, 3, 3])
backbone.fpn_output2.bias torch.Size([256])
backbone.fpn_lateral3.weight torch.Size([256, 512, 1, 1])
backbone.fpn_lateral3.bias torch.Size([256])
backbone.fpn_output3.weight torch.Size([256, 256, 3, 3])
backbone.fpn_output3.bias torch.Size([256])
backbone.fpn_lateral4.weight torch.Size([256, 1024, 1, 1])
backbone.fpn_lateral4.bias torch.Size([256])
backbone.fpn_output4.weight torch.Size([256, 256, 3, 3])
backbone.fpn_output4.bias torch.Size([256])
backbone.fpn_lateral5.weight torch.Size([256, 2048, 1, 1])
backbone.fpn_lateral5.bias torch.Size([256])
backbone.fpn_output5.weight torch.Size([256, 256, 3, 3])
backbone.fpn_output5.bias torch.Size([256])
backbone.bottom_up.stem.conv1.weight torch.Size([64, 3, 7, 7])
backbone.bottom_up.res2.0.shortcut.weight torch.Size([256, 64, 1, 1])
backbone.bottom_up.res2.0.conv1.weight torch.Size

In [10]:
if test_mode:
    my_mask_rcnn_model = MyMaskRCNN(cfg=cfg)
    my_mask_rcnn_model.load_from_pkl(MyMRCNN_PATH)
    for name, param in my_mask_rcnn_model.named_parameters():
        print(name, param.shape)

[MyMaskRCNN] 根据 cfg 创建空模型结构。
[MyMaskRCNN] 从 ../model/my_mask_mrcnn.pkl 加载完成。
backbone.fpn_lateral2.weight torch.Size([256, 256, 1, 1])
backbone.fpn_lateral2.bias torch.Size([256])
backbone.fpn_output2.weight torch.Size([256, 256, 3, 3])
backbone.fpn_output2.bias torch.Size([256])
backbone.fpn_lateral3.weight torch.Size([256, 512, 1, 1])
backbone.fpn_lateral3.bias torch.Size([256])
backbone.fpn_output3.weight torch.Size([256, 256, 3, 3])
backbone.fpn_output3.bias torch.Size([256])
backbone.fpn_lateral4.weight torch.Size([256, 1024, 1, 1])
backbone.fpn_lateral4.bias torch.Size([256])
backbone.fpn_output4.weight torch.Size([256, 256, 3, 3])
backbone.fpn_output4.bias torch.Size([256])
backbone.fpn_lateral5.weight torch.Size([256, 2048, 1, 1])
backbone.fpn_lateral5.bias torch.Size([256])
backbone.fpn_output5.weight torch.Size([256, 256, 3, 3])
backbone.fpn_output5.bias torch.Size([256])
backbone.bottom_up.stem.conv1.weight torch.Size([64, 3, 7, 7])
backbone.bottom_up.res2.0.shortcut.weight 

测试结束

现在考虑将其与CLIP融合

In [ ]:
class MyZeroShotOpenVocabularyDetector(nn.Module):
    def __init__(self, clip_path, my_mask_mrcnn_path, mrcnn_cfg, device):
        super().__init__()
        
        self.clip_model = CLIPModel.from_pretrained(clip_path).to(device)
        self.clip_processor = CLIPProcessor.from_pretrained(clip_path)
        
        self.my_mask_rcnn = MyMaskRCNN(cfg=mrcnn_cfg)
        self.my_mask_rcnn.load_from_pkl(my_mask_mrcnn_path)
        
    @torch.no_grad()
    def predict(self, dataset_dicts, meta_data=None, batch_size=1):
        """
        dataset_dicts: list of dicts, Detectron2 dataset format
        meta_data: MetadataCatalog.get(...) 对象，可选
        batch_size: int
        """
        results = []
        num_images = len(dataset_dicts)
        
        # 处理 batch
        for i in range(0, num_images, batch_size):
            batch = dataset_dicts[i:i+batch_size]
            
            # 将图片 tensor 移动到 device
            images = [d["image"].to(self.device) for d in batch]
            # Detectron2 预处理（pad to max size, normalize）
            images = self.my_mask_rcnn.preprocess_image(images)
            
            # backbone + rpn + roi_heads
            features = self.my_mask_rcnn.backbone(images.tensor)
            proposals, _ = self.my_mask_rcnn.proposal_generator(images, features)
            
            # ROI heads，得到 mask + box 修正
            # 因为你需要 box_predictor 修正后的 box
            detections, _ = self.my_mask_rcnn.roi_heads(images, features, proposals)
            
            # postprocess 每张图
            for det, inp in zip(detections, batch):
                # det 是 Instances 对象，包含:
                # det.pred_boxes, det.pred_masks, det.scores, det.pred_classes
                out_dict = {
                    "image_id": inp["image_id"],
                    "boxes": det.pred_boxes.tensor.cpu(),    # 修正后的 boxes [N,4]
                    "scores": det.scores.cpu(),
                    "classes": det.pred_classes.cpu(),
                    "masks": det.pred_masks.cpu()           # [N,H,W]
                }
                results.append(out_dict)
        
        return results
    